In [ ]:
%pip install ollama

In [16]:
# Loading the dataset
dataset = []
with open("data/cat-facts.txt", 'r', encoding='utf-8') as file:
    dataset = file.readlines()
    print(f'Loaded {len(dataset)} entries')

Loaded 150 entries


In [17]:
import ollama

EMBEDDING_MODEL = 'hf.co/CompendiumLabs/bge-base-en-v1.5-gguf'
LANGUAGE_MODEL = 'hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF'

# Each element in the VECTOR_DB will be a tuple (chunk, embedding)
# The embedding is a list of floats, for example: [0.1, 0.04, -0.34, 0.21, ...]
VECTOR_DB = []

def add_chunk_to_database(chunk):
  embedding = ollama.embed(model=EMBEDDING_MODEL, input=chunk)['embeddings'][0]
  VECTOR_DB.append((chunk, embedding))

In [28]:
for i, chunk in enumerate(dataset):
  add_chunk_to_database(chunk)
  # print(f'Added chunk {i+1}/{len(dataset)} to the database')
print(VECTOR_DB[0])

('On average, cats spend 2/3 of every day sleeping. That means a nine-year-old cat has been awake for only three years of its life.\n', [-0.035827987, -0.023908228, 0.04710774, -0.07914111, 0.037536673, -0.0128521, 0.07718161, 0.05329961, -0.013592441, -0.0026980033, -0.019847509, -0.0069340826, -0.059300855, 0.012656533, -0.045660704, 0.041647755, 0.086453356, 0.010321346, -0.032246, -0.029764228, 0.004843106, 0.027266791, -0.025671566, -0.00062146486, 0.052264057, -0.013294386, -0.011778823, -0.0027142977, 0.0032102594, -0.01180697, 0.038059242, -0.032774, -0.034357723, 0.006636201, 0.026008785, -0.035773687, -0.01358905, -0.033349123, 0.014084845, 0.033174273, -0.034342058, -0.012625629, -0.01879208, 0.015289299, -0.030113574, -0.016923776, -0.0018831983, -0.014748167, 0.025607035, 0.015993135, -0.0331077, 0.0026133861, 0.0040115737, -0.0065954635, -0.020896073, 0.03985064, 0.009599929, -0.0031136095, -0.00728591, 0.024220461, 0.044346593, 0.005362779, 0.0075635794, -0.014858337, -0

In [19]:
def cosine_similarity(a, b):
  dot_product = sum([x * y for x, y in zip(a, b)])
  norm_a = sum([x ** 2 for x in a]) ** 0.5
  norm_b = sum([x ** 2 for x in b]) ** 0.5
  return dot_product / (norm_a * norm_b)

In [20]:
def retrieve(query, top_n=3):
  query_embedding = ollama.embed(model=EMBEDDING_MODEL, input=query)['embeddings'][0]
  # temporary list to store (chunk, similarity) pairs
  similarities = []
  for chunk, embedding in VECTOR_DB:
    similarity = cosine_similarity(query_embedding, embedding)
    similarities.append((chunk, similarity))
  # sort by similarity in descending order, because higher similarity means more relevant chunks
  similarities.sort(key=lambda x: x[1], reverse=True)
  # finally, return the top N most relevant chunks
  return similarities[:top_n]

In [24]:
# Generation phrase
input_query = input('Ask me a question: ')
retrieved_knowledge = retrieve(input_query)

print('Retrieved knowledge: ')
for chunk, similarity in retrieved_knowledge:
    print(f'- (similarity: {similarity:.2f}) {chunk}')

# Generate prompt safely
context_text = '\n'.join([f' - {chunk}' for chunk, _ in retrieved_knowledge]) if retrieved_knowledge else "(No relevant context found.)"

instruction_prompt = f'''You are a helpful chatbot.
Use only the following pieces of context to answer the question. Don't make up any new information:
{context_text}
'''

Retrieved knowledge: 
- (similarity: 0.88) Cats sleep 16 to 18 hours per day. When cats are asleep, they are still alert to incoming stimuli. If you poke the tail of a sleeping cat, it will respond accordingly.

- (similarity: 0.82) On average, cats spend 2/3 of every day sleeping. That means a nine-year-old cat has been awake for only three years of its life.

- (similarity: 0.69) Cats spend nearly 1/3 of their waking hours cleaning themselves.



In [25]:
stream = ollama.chat(
  model=LANGUAGE_MODEL,
  messages=[
    {'role': 'system', 'content': instruction_prompt},
    {'role': 'user', 'content': input_query},
  ],
  stream=True,
)

# print the response from the chatbot in real-time
print('Chatbot response:')
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

Chatbot response:
According to the given information, on average, cats spent 2/3 of every day sleeping. This means that a nine-year-old cat had been awake for only three years out of its life. 

So, the number of hours per day they used to sleep would be:

9 years / 3 = 3 hours